#### ingresar a google drive

In [67]:
import os
import shutil
import tempfile
import csv
from datetime import datetime  # Agregamos la importación de datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargado {int(status.progress() * 100)}%.")

# Función para subir un archivo a Google Drive
def subir_archivo(file_id, upload_file_path):
    media = MediaFileUpload(upload_file_path, resumable=True)
    updated_file = drive_service.files().update(fileId=file_id, media_body=media).execute()
    print('Archivo actualizado en Google Drive:', updated_file.get('name'))

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Lista para almacenar los archivos modificados
archivos_modificados = []

# Obtener lista de archivos en la carpeta de Google Drive
results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
files = results.get('files', [])

# Iterar sobre los archivos
for file in files:
    file_id = file['id']
    file_name = file['name']
    download_path = os.path.join(temp_dir, file_name)

    # Descargar el archivo de Google Drive
    descargar_archivo(file_id, download_path)

    # Leer el archivo CSV y hacer modificaciones
    with open(download_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)
        for row_index, row in enumerate(rows):
            for col_index, cell in enumerate(row):
                if not cell:
                    rows[row_index][col_index] = "no definido"

    # Guardar los cambios en el archivo temporal sin la extensión _temp.csv
    download_path_temp = os.path.join(temp_dir, os.path.splitext(file_name)[0])
    with open(download_path_temp, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)

    archivos_modificados.append((file_id, download_path_temp))

# Subir los archivos modificados a Google Drive
for file_id, download_path_temp in archivos_modificados:
    subir_archivo(file_id, download_path_temp)

# Guardar la marca de tiempo de la última ejecución
# Puedes cambiar esto según cómo desees formatear la marca de tiempo
marca_de_tiempo_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
with open("ultima_ejecucion.txt", "w") as file:
    file.write(marca_de_tiempo_actual)


Descargado 100%.
Descargado 65%.
Descargado 100%.
Descargado 81%.
Descargado 100%.
Descargado 100%.
Descargado 100%.
Descargado 100%.
Archivo actualizado en Google Drive: BegInvFINAL12312016_Clean.csv
Archivo actualizado en Google Drive: PurchasesFINAL12312016_Clean.csv
Archivo actualizado en Google Drive: SalesFINAL12312016_Clean.csv
Archivo actualizado en Google Drive: InvoicePurchases12312016_Clean.csv
Archivo actualizado en Google Drive: 2017PurchasePricesDec_Clean.csv
Archivo actualizado en Google Drive: EndInvFINAL12312016_Clean.csv


#### cambio de datos nulos y actualizacion del archivo

In [68]:
# Función para listar las tablas en la base de datos
def listar_tablas():
    try:


# Define the connection string
        conn_str = (
            'DRIVER={ODBC Driver 17 for SQL Server};'
            'SERVER=JULIAN;'
            'DATABASE=Top-Drinks;'
            'Trusted_Connection=yes;'
        )

        # Establish a connection
        try:
            conn = pyodbc.connect(conn_str)
            print("Conexión exitosa")
            # Create a cursor
            cursor = conn.cursor()

            # Ahora puedes ejecutar comandos SQL usando el cursor
            # cursor.execute("SELECT * FROM tu_tabla")
            # rows = cursor.fetchall()
            # for row in rows:
            #     print(row)
        except pyodbc.Error as e:
            print("Error en la conexión a SQL Server:", e)


        # Now you can execute SQL commands using the cursor

        # Obtener la lista de tablas
        cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")
        tablas = cursor.fetchall()

        print("Tablas en la base de datos:")
        for tabla in tablas:
            print(tabla[0])

        cursor.close()
        conn.close()
    except Exception as e:
        print(f'Error en la conexión a SQL Server: {str(e)}')

# Llamar a la función para listar las tablas
listar_tablas()

    


Conexión exitosa
Tablas en la base de datos:
2017PurchasePricesDec_Clean
BegInvFINAL12312016_Clean


In [87]:
import os
import csv
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse  # Agregado para construir la cadena de conexión

# Definir la ruta al archivo JSON de credenciales de Google Drive
credentials_file = 'data-424019-28bfddebf741.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(credentials_file)

# Crear un cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive donde se encuentran los archivos CSV
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Directorio base donde se guardarán los archivos CSV descargados
base_directory = '/ruta/a/tu/directorio/base'
csv_folder_name = 'archivos_csv_google_drive'
local_directory = os.path.join(base_directory, csv_folder_name)

# Función para descargar archivos CSV de Google Drive
def download_csv_files_from_drive(folder_id, local_directory):
    # Eliminar archivos CSV antiguos en el directorio local
    for file in os.listdir(local_directory):
        if file.endswith('.csv'):
            os.remove(os.path.join(local_directory, file))
            print(f"Removed old CSV file: {file}")
    
    # Descargar los nuevos archivos CSV desde Google Drive
    results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
    files = results.get('files', [])
    for file in files:
        file_id = file['id']
        file_name = file['name']
        download_path = os.path.join(local_directory, file_name)
        request = drive_service.files().get_media(fileId=file_id)
        with open(download_path, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Downloaded {int(status.progress() * 100)}% of {file_name}")

# Función para cargar datos desde archivos CSV a una base de datos SQL Server
def load_csv_data_to_sql(csv_directory, conn_str):
    files = os.listdir(csv_directory)
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(csv_directory, file)
            table_name = os.path.splitext(file)[0]  # Utilizar el nombre del archivo CSV como nombre de la tabla
            
            # Define el delimitador predeterminado
            delimiter = ';'
            
            # Verifica si el nombre del archivo es uno de los archivos específicos
            special_tables = ['BegInvFINAL12312016_Clean', 'EndInvFINAL12312016_Clean']
            if table_name in special_tables:
                delimiter = ','  # Cambia el delimitador a ',' para estos archivos específicos
                
            # Define una función de redondeo personalizada
            def custom_round(value):
                if isinstance(value, float):
                    return round(value, 2)
                return value
            
            # Leer el archivo CSV con el delimitador correcto y aplicar la función de redondeo personalizada
            df = pd.read_csv(file_path, delimiter=delimiter, converters={i: custom_round for i in range(10)})  # ajusta el rango 10
            
            # Crear el motor de conexión a la base de datos
            engine = create_engine(conn_str)
            
            # Cargar los datos en SQL Server
            df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
            print(f"Data from {file} loaded into SQL table {table_name}")

# Ejemplo de uso
if __name__ == "__main__":
    # Crear el directorio local si no existe
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)

    # Descargar archivos CSV de Google Drive a un directorio local
    download_csv_files_from_drive(folder_id, local_directory)
    
    # Construir la cadena de conexión para SQLAlchemy
    params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=JULIAN;DATABASE=Top-Drinks;Trusted_Connection=yes;")
    conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
    
    # Cargar datos desde archivos CSV a una base de datos SQL Server
    load_csv_data_to_sql(local_directory, conn_str)



Removed old CSV file: 2017PurchasePricesDec_Clean.csv
Removed old CSV file: BegInvFINAL12312016_Clean.csv
Removed old CSV file: EndInvFINAL12312016_Clean.csv
Removed old CSV file: InvoicePurchases12312016_Clean.csv
Removed old CSV file: PurchasesFINAL12312016_Clean.csv
Removed old CSV file: SalesFINAL12312016_Clean.csv
Downloaded 100% of BegInvFINAL12312016_Clean.csv
Downloaded 65% of PurchasesFINAL12312016_Clean.csv
Downloaded 100% of PurchasesFINAL12312016_Clean.csv
Downloaded 81% of SalesFINAL12312016_Clean.csv
Downloaded 100% of SalesFINAL12312016_Clean.csv
Downloaded 100% of InvoicePurchases12312016_Clean.csv
Downloaded 100% of 2017PurchasePricesDec_Clean.csv
Downloaded 100% of EndInvFINAL12312016_Clean.csv
Data from 2017PurchasePricesDec_Clean.csv loaded into SQL table 2017PurchasePricesDec_Clean
Data from BegInvFINAL12312016_Clean.csv loaded into SQL table BegInvFINAL12312016_Clean
Data from EndInvFINAL12312016_Clean.csv loaded into SQL table EndInvFINAL12312016_Clean
Data from 

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)